In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


repo_id = "/root/share/LLama2/llama-2-13b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(repo_id, device_map='auto', torch_dtype=torch.float16, load_in_8bit=False)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(repo_id, use_fast=False)


/root/miniconda3/envs/glm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-11 13:27:37,762] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [01:08<00:00, 22.81s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [5]:
tokenizer.pad_token = tokenizer.eos_token
input_ids = tokenizer(['<s>Human: Introduce the history of Shenzhen\n</s><s>Assistant: '], return_tensors="pt",add_special_tokens=False).input_ids.to('cuda')        
generate_input = {
    "input_ids":input_ids,
    "max_new_tokens":512,
    "do_sample":True,
    "top_k":50,
    "top_p":0.95,
    "temperature":0.3,
    "repetition_penalty":1.3,
    "eos_token_id":tokenizer.eos_token_id,
    "bos_token_id":tokenizer.bos_token_id,
    "pad_token_id":tokenizer.pad_token_id
}
generate_ids  = model.generate(**generate_input)
text = tokenizer.decode(generate_ids[0])
print(text)

<s> Human: Introduce the history of Shenzhen
</s><s> Assistant:  Sure, I'd be happy to help! Here is a brief overview of the history of Shenzhen:

Shenzhen has been inhabited for thousands of years and was once known as "Binhai" or "Fisher Girl's Head." In the early 1980s, it was designated as China's first special economic zone (SEZ) and underwent rapid development. Today, Shenzhen is one of the most prosperous cities in China and is home to many major companies such as Huawei Technologies Co., Ltd. and Tencent Holdings Limited. The city also boasts numerous universities and research institutions, including Tsinghua University-Shenzhen Graduate School and the Chinese Academy of Sciences. Additionally, Shenzhen is famous for its vibrant culture scene with various museums like OCT-LOFT Creative Culture Park and Window of the World theme park which showcases miniature versions of world landmarks. Would you like me to elaborate on any specific aspect?  </s>


In [3]:
from bigmodelvis import Visualization

Visualization(model).structure_graph();

root
├── model (LlamaModel)
│   ├── embed_tokens (Embedding) weight:[32000, 5120]
│   ├── layers (ModuleList)
│   │   └── 0-39(LlamaDecoderLayer)
│   │       ├── self_attn (LlamaAttention)
│   │       │   └── q_proj,k_proj,v_proj,o_proj(Linear) weight:[5120, 5120]
│   │       ├── mlp (LlamaMLP)
│   │       │   ├── gate_proj,up_proj(Linear) weight:[13824, 5120]
│   │       │   └── down_proj (Linear) weight:[5120, 13824]
│   │       └── input_layernorm,post_attention_layernorm(LlamaRMSNorm) weight:[5120]
│   └── norm (LlamaRMSNorm) weight:[5120]
└── lm_head (Linear) weight:[32000, 5120]